# Introduction

**Objective:**  
Develop a classification model to identify high-risk patients in the ICU and assign a triage tag. 

**Data**  
Dataset was developed as part of MIT's GOSSIS community initiative, with privacy certification from the Harvard Privacy Lab. https://www.kaggle.com/c/widsdatathon2020/data

**Triage Protocal**
* Green: (wait) are reserved for the "walking wounded" who will need medical care at some point, after more critical injuries have been treated.
* Yellow: (observation) for those who require observation (and possible later re-triage). Their condition is stable for the moment and, they are not in immediate danger of death. These victims will still need hospital care and would be treated immediately under normal circumstances.
* Red: (immediate) are used to label those who cannot survive without immediate treatment but who have a chance of survival.
* Black: (expectant) are used for the deceased and for those whose injuries are so extensive that they will not be able to survive given the care that is available.
Note: this model will not identify black tags. Within the red group, physicians should assign additional tagging if needed. 

**Metrics**  
The goal is to flag anyone entering the ICU who is at risk. With that, recall is the main metric although AUC-ROC and precision will be used as secondary considerations. 


# Load Data & Packages

In [ ]:
from load_data import access_aws, load_and_info, import_files, basic_data_info
from eda import create_cat_df, pairplotting, selecting_cols, exploring_y
from data_cleaning import clean_data
from model_dev import (tts, test_model_framework, forced_grid_search, 
                        make_confusion_matrix, final_model)
import assigning_triage_levels

from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier, VotingClassifier, 
                              AdaBoostClassifier, BaggingRegressor)
from mlxtend.classifier import StackingClassifier

from ipywidgets import interactive, FloatSlider

import numpy as np
import pandas as pd

## Import Dataset

In [ ]:
#Pull the full icu_mortality data from AWS for EDA and feature engineering
query_all_data = '''SELECT * FROM icu_mortality_table'''

#replace host IP each time instance is restarted
aws_access = (aws_code())
raw_data = access_aws('52.14.254.66', 'icu_mortality',query_all_data)

In [ ]:
# call function to view basic information about the data
basic_data_info(raw_data)

## Import Data Dictionary

In [ ]:
#Pull data_dict from AWS to help understand the raw_data file
query_data_dict = '''SELECT * FROM icu_data_dict_table'''
data_dict = access_aws('52.14.254.66', 'icu_mortality',query_data_dict)

In [ ]:
data_dict.info()

# Exploratory Data Analysis

In [ ]:
# To view dataset in smaller batches, break the full set into smaller groups based on the 
# category that the column falls under occording to the data dictionary

demo_data = create_cat_df(raw_data, 'demographic', data_dict, demo=True)
apache_covariate_data = create_cat_df(raw_data, 'APACHE covariate', data_dict)
vitals_data = create_cat_df(raw_data, 'vitals', data_dict)
labs_data = create_cat_df(raw_data, 'labs', data_dict)
labs_blood_gas_data = create_cat_df(raw_data, 'labs blood gas', data_dict)
apache_prediction_data = create_cat_df(raw_data, 'APACHE prediction', data_dict)
apache_comorbidity_data = create_cat_df(raw_data, 'APACHE comorbidity', data_dict)

## Observe Dependent Variable - Hospital Survival

In [ ]:
def exploring_y(data):
    """Reveals a few key characterstics of the y_value, hospital_death."""
    print("Unique hospital death values:\n", data.hospital_death.unique())
    print("\nNumber of each y value:\n", data.hospital_death.value_counts())
    print("\nPercentage of survivors:\n", data.hospital_death.value_counts(normalize=True)*100)
    return

exploring_y(raw_data)

## Observe Features

In [ ]:
def selecting_cols(data):
    """Takes in a dataframe and calculates the difference in mean between
    those that survive and do not survive for each column. Returns a dataframe
    of only columns with signfificant differentiation between the means as well
    as a list of the columns included in the Dataframe. Differentiation is
    considered signficant if mean(1)/mean(0) > 1.5 or <.75."""

    selected_cols = []
    dif_in_means = data.groupby('hospital_death').mean().iloc[0,:] / data.groupby('hospital_death').mean().iloc[1,:]
    high_end = dif_in_means.index[dif_in_means > 1.5].to_list()
    low_end = dif_in_means.index[dif_in_means < .75].to_list()
    selected_cols += high_end
    selected_cols += low_end
    selected_cols.append('hospital_death')
    if len(selected_cols) > 1:
        selected_data = data[selected_cols]
        return selected_data, selected_cols
    else:
        print("No signficant features. Additional exploration needed.")
        return dif_in_means, 0

### 7-Figure Summary Per Category

In [ ]:
demo_data.describe()

In [ ]:
apache_covariate_data.describe()

In [ ]:
vitals_data.describe()

In [ ]:
labs_data.describe()

In [ ]:
labs_blood_gas_data.describe()

In [ ]:
apache_prediction_data.describe()

In [ ]:
apache_comorbidity_data.describe()

### Determinging Significant Features

In order to determing which features to begin building the model with, view the difference in values for each classification. If the difference in value for y = 1 and y = 0 is > 1.5 or < .75, feature will be considered significant. 

In [ ]:
def selecting_cols(data):
    """Takes in a dataframe and calculates the difference in mean between
    those that survive and do not survive for each column. Returns a dataframe
    of only columns with signfificant differentiation between the means as well
    as a list of the columns included in the Dataframe. Differentiation is
    considered signficant if mean(1)/mean(0) > 1.5 or <.75."""

    selected_cols = []
    dif_in_means = data.groupby('hospital_death').mean().iloc[0,:] / data.groupby('hospital_death').mean().iloc[1,:]
    high_end = dif_in_means.index[dif_in_means > 1.5].to_list()
    low_end = dif_in_means.index[dif_in_means < .75].to_list()
    selected_cols += high_end
    selected_cols += low_end
    selected_cols.append('hospital_death')
    if len(selected_cols) > 1:
        selected_data = data[selected_cols]
        return selected_data, selected_cols
    else:
        print("No signficant features. Additional exploration needed.")
        return dif_in_means, 0

In [ ]:
demo_df, demo_df_cols = selecting_cols(demo_data)

In [ ]:
apach_cov_df, apach_cov_df_cols = selecting_cols(apache_covariate_data)

In [ ]:
vitals_df, vitals_df_cols = selecting_cols(vitals_data)

In [ ]:
labs_df, labs_df_cols = selecting_cols(labs_data)

In [ ]:
labs_bg_df, labs_bg_df_cols = selecting_cols(labs_blood_gas_data)

In [ ]:
apach_pred_df, apach_pred_df_cols = selecting_cols(apache_prediction_data)

In [ ]:
apach_com_df, apach_com_df_cols = selecting_cols(apache_comorbidity_data)

# Data Cleaning

In [ ]:
def drop_null(data):
    """Drops any columns with > 50% of the columns missing. From remaining columsn,
    drops all rows with null values."""

    cols_with_many_null = (data.isna().sum() / len(data)).sort_values(ascending = False)
    cols_with_many_null = cols_with_many_null.index[cols_with_many_null > 0.50]
    print('There are %d columns with more than 50%% missing values' % len(cols_with_many_null))

    #drop columns with > 50% null values
    data = data.drop(cols_with_many_null, axis=1)
    print("Columns dropped")

    data = data.dropna()
    print("Rows with nulls dropped.")

    return data

def clean_data(data):
    #remove values of icu days, hospital death prob, and icu death prob that are less than 0
    #as these values are not possible
    data1 = data[data.pre_icu_los_days >= 0]
    data2 = data[data.apache_4a_hospital_death_prob >= 0]
    data3 = data[data.apache_4a_icu_death_prob >= 0]

    #fill in columns with many nulls to indicate weather test was performed or not
    data3.h1_lactate_max = data3.h1_lactate_max.fillna(0)
    data3.h1_lactate_max = data3.h1_lactate_max.apply(lambda x: 1 if x != 0 else x)
    data3.h1_inr_max = data3.h1_inr_max.fillna(0)
    data3.h1_inr_max = data3.h1_inr_max.apply(lambda x: 1 if x != 0 else x)
    data3.bilirubin_apache = data3.bilirubin_apache.fillna(0)
    data3.bilirubin_apache = data3.bilirubin_apache.apply(lambda x: 1 if x != 0 else x)

    #replace nulls in hospital_admit_source with 'unkown'
    data3.hospital_admit_source = data3.hospital_admit_source.fillna('unkown')

    #remove additional nulls from dataset
    cleaned_data = drop_null(data3)

    return cleaned_data

In [ ]:
cleaned_data = clean_data(raw_data)

In [ ]:
cleaned_data.info()

In [ ]:
#Creating data groups by category with clean data
demo_data_clean = create_cat_df(cleaned_data, 'demographic', data_dict, demo=True)
apache_covariate_data_clean = create_cat_df(cleaned_data, 'APACHE covariate', data_dict)
vitals_data_clean = create_cat_df(cleaned_data, 'vitals', data_dict)
labs_data_clean = create_cat_df(cleaned_data, 'labs', data_dict)
labs_blood_gas_data_clean = create_cat_df(cleaned_data, 'labs blood gas', data_dict)
apache_prediction_data_clean = create_cat_df(cleaned_data, 'APACHE prediction', data_dict)
apache_comorbidity_data_clean = create_cat_df(cleaned_data, 'APACHE comorbidity', data_dict)

In [ ]:
#reselecting columns based on cleaned data for significant categories
demo_df, demo_df_cols = selecting_cols(demo_data_clean)
apach_cov_df, apach_cov_df_cols = selecting_cols(apache_covariate_data_clean)
labs_df, labs_df_cols = selecting_cols(labs_data_clean)
apach_pred_df, apach_pred_df_cols = selecting_cols(apache_prediction_data_clean)
apach_com_df, apach_com_df_cols = selecting_cols(apache_comorbidity_data_clean)

# Feature Engineering

In a separate notebook, combinations of the columns selected above were tested in logistic regression and random forest models. Through that process, the columns and engineered features below were selected as top performers. 

In [ ]:
# Selecting columns based on above EDA
features = apach_pred_df_cols + demo_df_cols + apach_cov_df_cols + cat_cols_to_keep

# In above category separation, hospital_death, the dependent variable, is included in all of the 
# categories. It therefore is removed twice to ensure only one copy of hospital_death is in the 
# dataset
features.remove('hospital_death')
features.remove('hospital_death')

# create new dataframe with these final features
model_df = cleaned_data[features]

In [ ]:
# Additional feature engineering to increase signal and decrease noise in weaker features
model_df.pre_icu_los_days = model_df.pre_icu_los_days.apply(lambda x: x**2)
model_df['bun_creatinine'] = model_df.bun_apache * combo5_df.creatinine_apache
model_df = model_df.drop(['bun_apache', 'creatinine_apache'], axis=1)

In [ ]:
# save cleaned, engineered dataframe for later use
model_df.to_csv('model_df.csv')

# Model Development

In a separate notebook, a full exploration of classification models including Decision Trees, Logistic Regression, K-Nearest Neighbors (KNN), and Support Vector Machines was conducted. Random Forests performed best with the highest recall, auc, and precision scores. 

## Train Test Split

In [ ]:
def dummy_data(data):
    """Creates dummy variables for each categorical column."""

    cat_cols = [col for col in data.columns if (data[col].dtype == 'O')]
    dummied_data = pd.get_dummies(data, drop_first=True, columns=cat_cols)
    print("Dummy variables created.")
    return dummied_data

def create_x_y(data):
    """Creates X and y variables where y = 'hospital_death' column and
    X = remaining columns."""

    X, y = data.drop('hospital_death', axis=1), data['hospital_death']
    return X, y


def tts(data):
    """Splits the dataset into train and test sets. Calls create_x_y to
    split X and y, Calls dummy_data to turn any categorical columns into
    dummy variables."""
    dummied_data = dummy_data(data)

    X, y = create_x_y(dummied_data)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, stratify = y)
    print("Dataset split into train and test sets.\n")
    print("X_train shape:", X_train.shape)
    print("X_train shape:", y_train.shape)
    print("X_test shape:", X_test.shape)
    print("y_test shape:", y_test.shape)

    return X_train, X_test, y_train, y_test

In [ ]:
X_train_tuning, X_test_tuning, y_train_tuning, y_test_tuning = tts(model_df)

## Execution

In [ ]:
def roc_curve_plotting(fpr, tpr, auc_roc):
    """Takes in fpr, tpr, and auc_roc to produce an ROC curve for all
    validation sets."""

    plt.figure(figsize=(5,5))
    lw = 2
    for i in range(len(fpr)):
        plt.plot(fpr[i], tpr[i],
                 lw=lw, label='ROC curve (area = %0.2f)' % auc_roc[i])
        plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
    plt.show()
    return

def scaling_data(X_tr, X_val):
    """Separates train and validation datasets into categorical and numerical columns.
    Scales numerical columns using StandardScaler"""

    #separating numerical columns in train and validation sets
    num_cols = [col for col in X_tr.columns if (X_tr[col].dtype == float)]
    X_tr_num, X_tr_not_num = X_tr[num_cols], X_tr.drop(num_cols, axis=1)
    X_val_num, X_val_not_num = X_val[num_cols], X_val.drop(num_cols, axis=1)

    #scaling numerical columns only
    ss = StandardScaler()
    X_tr_scaled = ss.fit_transform(X_tr_num)
    X_val_scaled = ss.transform(X_val_num)

    #rejoining scaled columns with non-scaled columns
    X_tr = np.concatenate((X_tr_scaled, X_tr_not_num), axis=1)
    X_val = np.concatenate((X_val_scaled, X_val_not_num), axis=1)

    return X_tr, X_val

# generic model framework used to test a variety of models with a consistent function

def test_model_framework(X_train, y_train, model, scaling=False, smote=False, rand=False):
    """Within the 80% train data, splits the model into 5 folds for cross-validation testing.
    If model being implemented requires scaling, allows scaling function to be called.
    Also enables smote or random oversampling to be called."""

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 71)
    print("Kfolds created.")

    recall, precision, accuracy, matrix, roc_auc = [], [], [], [], []
    fpr, tpr = [], []
    p, t, pr_thresholds = [], [], []

    # all scaling and oversampling must be performed within the k-fold cross validation to avoid
    # data leakage
    for train_ind, val_ind in kf.split(X_train,y_train):

        X_tr, y_tr = X_train.iloc[train_ind], y_train.iloc[train_ind]
        X_val, y_val = X_train.iloc[val_ind], y_train.iloc[val_ind]

        #standard scaling
        if scaling:
            X_tr, X_val = scaling_data(X_tr, X_val)

        if smote:
            ros = SMOTE(random_state=19)
            X_tr, y_tr = ros.fit_sample(X_tr, y_tr)

        if rand:
            ros = RandomOverSampler(random_state=19)
            X_tr, y_tr = ros.fit_sample(X_tr, y_tr)

        mod = model
        mod.fit(X_tr, y_tr)
        prediction_prob = mod.predict_proba(X_val)
        prediction = mod.predict(X_val)
        
        # for each k-fold, append recall, precision, accuracy, and confusion matrix scores to
        # lists above
        recall.append(recall_score(y_val, prediction, average='binary', pos_label=1))
        precision.append(precision_score(y_val, prediction, average='binary', pos_label=1))
        accuracy.append(accuracy_score(y_val, prediction))
        matrix.append(confusion_matrix(y_val, prediction))

        # For each k-fold, calculate roc_curve parameters and append to lists above 
        fpr_ex, tpr_ex, thresholds_ex = roc_curve(y_val, prediction_prob[:,1], pos_label=1)
        fpr.append(fpr_ex)
        tpr.append(tpr_ex)
        roc_auc.append(roc_auc_score(y_val, prediction_prob[:,1]))

    # After cross validation, print out the mean scores
    print("Confusion matrix:\n", (matrix[0] + matrix[1] + matrix[2] + matrix[3] + matrix[4])/5)
    print("Mean recall: ", np.mean(recall))
    print("Mean precision: ", np.mean(precision))
    print("Mean accuracy: ", np.mean(accuracy))

    # plot the mean roc curve
    roc_curve_plotting(fpr, tpr, roc_auc)

    # returns the model so it can be used later on for any plotting
    return mod

In [ ]:
def forced_grid_search(X_train, y_train):
    """Mimics grid search but calls test_model_framework function to include oversampling within
    the k-fold."""
    for n in [100,200,500]:
        for m in [5,10,50,100,500]:
            for z in [5,10,50,100,500]:
                print(f"For n_estimators = {n}, max_depth = {m}, min_samples_leaf={z}")
                rf_tuning = RandomForestClassifier(n_estimators=n,
                                                max_depth=m,
                                                min_samples_leaf=10)

                tuning_model_run = test_model_framework(X_train, y_train, rf_tuning,
                                                scaling=False, smote=False, rand=True)

    return

In [ ]:
# test a variety of different random forest conditions to tune model
forced_grid_search(X_train_tuning, y_train_tuning)

## Ensembling

In [ ]:
#Soft voting classifier with logistic regression, random forest, and knn
model_list = [('rf', RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_split=2,min_samples_leaf=10)), 
              ('lr', LogisticRegression(solver='liblinear', C=.01)),
              ('knn', KNeighborsClassifier(n_neighbors=100))]

voting_classifer = feature_testing_simple_model(X_train_fe, y_train_fe, VotingClassifier(estimators=model_list, 
                                                                      voting='soft',
                                                                      n_jobs=-1), 
                             scaling=True, smote=False, rand=True)

In [ ]:
# stacked classifier with same 3 models and logistic regression metaclassifier

models = [RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_split=2,min_samples_leaf=10), 
            LogisticRegression(solver='liblinear', C=.01),
            KNeighborsClassifier(n_neighbors=100)]

stacked = StackingClassifier(classifiers=models, 
                             meta_classifier=LogisticRegression(C=.01), 
                             use_probas=True, use_features_in_secondary=False)

stacking_classifier = feature_testing_simple_model(X_train_fe, y_train_fe, stacked, scaling=True, smote=False, rand=True)

In [ ]:
# stacked classifier with random forest and logistic regression. 
# Random forest metaclassifier with reusing features in secondary model run. 

models = [RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_split=2,min_samples_leaf=10), 
            LogisticRegression(solver='liblinear', C=.01)]

stacked = StackingClassifier(classifiers=models, 
                             meta_classifier=RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_split=2,min_samples_leaf=10), 
                             use_probas=True, use_features_in_secondary=True)

stacking_classifier = feature_testing_simple_model(X_train, y_train, stacked, scaling=True, smote=False, rand=True)

## Threshold Adjustments

In [ ]:
# Use best performing parameters to create a final model

def final_model(X_train, X_test, y_train, y_test):
    """Takes in train test split values, trains a random forest model with best-performing 
    hyperparameters and prints out model metrics."""
    
    ros = RandomOverSampler(random_state=19)
    X_train, y_train = ros.fit_sample(X_train, y_train)

    # best performing metrics during testing
    model = RandomForestClassifier(n_estimators = 100,
                                   max_depth = 5,
                                   min_samples_leaf=100)
    model.fit(X_train, y_train)
    prediction = prediction = model.predict(X_test)

    print("Model Recall: ", recall_score(y_test, prediction, average='binary', pos_label=1))
    print("Model Precision: ", precision_score(y_test, prediction))
    print("Model Accuracy: ", accuracy_score(y_test, prediction))

    return model

In [ ]:
# create interactive threshold adjuster to observe changes in the confusion matrix

def make_confusion_matrix(X_test, y_test, model, threshold):
    # Predict class 1 if probability of being in class 1 is greater than threshold
    # (model.predict(X_test) does this automatically with a threshold of 0.5)
    y_predict = (model.predict_proba(X_test)[:, 1] >= threshold)
    icu_confusion = confusion_matrix(y_test, y_predict)
    plt.figure(dpi=80)
    sns.heatmap(icu_confusion, cmap=plt.cm.Blues, annot=True, square=True, fmt='d',
           xticklabels=['Surviving', 'Not-surviving'],
           yticklabels=['Surviving', 'Not-surviving']);
    plt.xlabel('prediction')
    plt.ylabel('actual')
    recall = icu_confusion[1][1] / (icu_confusion[1][1] + icu_confusion[1][0])
    precision = icu_confusion[1][1] / (icu_confusion[1][1] + icu_confusion[0][1])
    print('Recall: ', recall)
    print('Precision: ', precision)
    th_prec = pd.DataFrame({'threshold': threshold, 'prediction': y_predict})

    return

interactive(lambda threshold: make_confusion_matrix(X_test, y_test, final_model, threshold), threshold=(0.0,1.0,0.02))

# Append Prediction Column To Dataset

In order to create a Tableu dashboard that shows triage levels in the ICU for patient administration and physicians, append a predictions column back to the origonal dataset.

In [ ]:
def f(row):
    if row['flag'] <= .3:
        val = 'G'
    elif .3 < row['flag'] <= .7:
        val = 'Y'
    elif .7 < row['flag'] <= 1:
        val = 'R'
    return val

def add_flags_to_original_dataset(model, X_test, original_data):
    """Adds flags back to the original dataset without feature engineered
    columns for better visualization."""
    X_test_indices = X_test.index
    X_test_indices = [ind for ind in X_test_indices]
    X_test_from_original = original_data.loc[X_test_indices,:]
    X_test_from_original_flagged = pd.concat((X_test_from_original.reset_index(), pred_proba_df), axis=1)
    X_test_from_original_flagged = X_test_from_original_flagged.rename(columns={'index':'patient_id'})

    X_test_from_original_flagged['flag'] = X_test_from_original_flagged.pred_proba
    X_test_from_original_flagged['flag'] = X_test_from_original_flagged.apply(f, axis=1)

    X_test_proba_appended.to_csv('original_data_with_flag.csv')

    return X_test_proba_appended

original_data_with_flags = create_flag_column(final_model, X_test, raw_data)